In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/nfshome/jc9033/.conda/envs/py3/bin/python3.7'

from pyspark import SparkContext
sc = SparkContext('local', 'pyspark')

from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [2]:
sc

<SparkContext master=local appName=pyspark>

In [3]:
i = "2015"
path = "data_tickets/tickets_" + i + ".csv"
temp = sc.textFile(path).cache()
print(list(enumerate(temp.first().split(','))))

[(0, 'Summons Number'), (1, 'Plate ID'), (2, 'Registration State'), (3, 'Plate Type'), (4, 'Issue Date'), (5, 'Violation Code'), (6, 'Vehicle Body Type'), (7, 'Vehicle Make'), (8, 'Issuing Agency'), (9, 'Street Code1'), (10, 'Street Code2'), (11, 'Street Code3'), (12, 'Vehicle Expiration Date'), (13, 'Violation Location'), (14, 'Violation Precinct'), (15, 'Issuer Precinct'), (16, 'Issuer Code'), (17, 'Issuer Command'), (18, 'Issuer Squad'), (19, 'Violation Time'), (20, 'Time First Observed'), (21, 'Violation County'), (22, 'Violation In Front Of Or Opposite'), (23, 'House Number'), (24, 'Street Name'), (25, 'Intersecting Street'), (26, 'Date First Observed'), (27, 'Law Section'), (28, 'Sub Division'), (29, 'Violation Legal Code'), (30, 'Days Parking In Effect    '), (31, 'From Hours In Effect'), (32, 'To Hours In Effect'), (33, 'Vehicle Color'), (34, 'Unregistered Vehicle?'), (35, 'Vehicle Year'), (36, 'Meter Number'), (37, 'Feet From Curb'), (38, 'Violation Post Code'), (39, 'Violatio

## trial

street name

In [4]:
#distinct
def mapper1(partitionID, rows):
    if partitionID==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        yield(fields[24],fields[21])

In [ ]:
%%time
i = "2015"
path = "data_tickets/tickets_" + i + ".csv"
rdd = sc.textFile(path)\
         .mapPartitionsWithIndex(mapper1)\
         .distinct()      

In [ ]:
%%time
rdd.take(5)

2

In [ ]:
#distinct
def mapper1(partitionID, rows):
    if partitionID==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        yield((fields[4], fields[24],fields[21]),1)

def mapper2(partition):
    for element in partition:
        yield(element[0][0], element[0][1], element[0][2], element[1])

In [ ]:
%%time
i = "2015"
path = "data_tickets/tickets_" + i + ".csv"
rdd = sc.textFile(path)\
         .mapPartitionsWithIndex(mapper1)\
         .groupByKey().mapValues(sum)\
         .mapPartitions(mapper2)

In [ ]:
%%time
rdd.take(5)

# Cleaning & Integrating

In [5]:
%%time
for i in range(2014,2020):
    print(i)
    path = "data_tickets/tickets_" + str(i) + ".csv"
    if i == 2014:
        rdd = sc.textFile(path)\
                 .mapPartitionsWithIndex(mapper1)\
                 .distinct()      
    else:
        temp = sc.textFile(path)\
                 .mapPartitionsWithIndex(mapper1)\
                 .distinct()
        rdd = rdd.union(temp).distinct()
    rdd.take(5)

2014
2015
2016
2017
2018
2019
CPU times: user 264 ms, sys: 104 ms, total: 368 ms
Wall time: 9min 42s


### Output CSV

In [6]:
%%time
from pyspark import SparkContext
df = spark.createDataFrame(rdd, ['street_name', 'borough'])

CPU times: user 288 ms, sys: 80 ms, total: 368 ms
Wall time: 2.9 s


In [7]:
df.show()

+--------------------+-------+
|         street_name|borough|
+--------------------+-------+
|        WEST 58TH ST|       |
|             E 82 ST|     NY|
|         SHARON LANE|      R|
|          PARKER AVE|     BX|
|         CORTANA AVE|     BX|
|    E/S DOUGULAS AVE|     BX|
|      C/O N/E JEROME|     BX|
|          HUBERT AVE|     BX|
|          KENNEDY CR|      Q|
| E MOSHULO PKY SOUTH|     BX|
|           W 41ST ST|     NY|
|            21st Ave|      Q|
|        S/O W 174 ST|     BX|
|       S/W C/O 45 ST|      K|
|             E 5  ST|     NY|
|      W/S CHELSEA RD|      R|
|             C/O 193|      Q|
|    130 COURT STREET|      K|
|      C/O EDSALL AVE|     BX|
|N/N/C/O RESERVOIR AV|     BX|
+--------------------+-------+
only showing top 20 rows



In [8]:
%%time
df.coalesce(1).write.format('com.databricks.spark.csv').options(header='true').save('output_tickets_street_name')

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 18.1 s


###  Check file

In [2]:
filename = ! ls output_tickets_street_name | grep .csv
filename

['part-00000-16a3a53f-caa1-4c35-be0c-7a9ae32dbe71-c000-Copy1.csv',
 'tickets_street_name.csv']

In [4]:
import pandas as pd
pddf = pd.read_csv('output_tickets_street_name/'+filename[1], error_bad_lines=False)#.fillna(0)
print(pddf.shape)

b'Skipping line 109119: expected 2 fields, saw 159\n'


(358602, 2)


In [5]:
pddf.head()

,street_name,borough
0,WEST 58TH ST,NaN
1,E 82 ST,NY
2,SHARON LANE,R
3,PARKER AVE,BX
4,CORTANA AVE,BX


In [6]:
pddf.tail()

,street_name,borough
358597,S/O SHEFFIELD AVENUE,K
358598,GULF ST,R
358599,E/S LORIMER S/O BAYA,K
358600,ST PAUL S PLAZA,K
358601,RIDGEOOD AVE,K


In [7]:
len(pddf['street_name'].unique())

300966

In [13]:
pddf[10000:]

,street_name,borough
10000,ISABELLA AVE,R
10001,S/E CORNER OF 65 ST,K
10002,56TH,Q
10003,S/O 2275 SANDSON AV,BX
10004,N/S 32 AV,Q
10005,C/O CROTONA AVE,BX
10006,WATCHOGUE RD,R
10007,I/O 112 AV,Q
10008,83RD STREET,Q
10009,HIGHLAND BLVD 500FT,NaN
